In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys
import copy
import pickle
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras.api._v2.keras as keras
import keras.backend as K
from keras.optimizers.legacy import Adam
from keras.optimizers import SGD
from keras import callbacks
from keras.models import Model
from keras.layers import Layer, Dense, Input, Activation, Lambda, BatchNormalization, Conv1D, SpatialDropout1D, add, GlobalAveragePooling1D, LSTM, Dense, concatenate, TimeDistributed, Bidirectional, Dropout, Embedding, Attention, MultiHeadAttention, LayerNormalization, Flatten, Concatenate
from keras.activations import sigmoid
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score
import tensorflow as tf
from data_load import load_data
from basemodel import BaseModel
from train import cross_val_training

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Load data
train_data, train_label, test_data, test_label, train_text, train_audio, train_visual, test_text, test_audio, test_visual = load_data()

Number of training samples:  120
Number of testing samples:  31
Max length of sequences:  110

Train text shape: 120 samples, 110 timesteps, 100 features
Train audio shape: 120 samples, 110 timesteps, 100 features
Train visual shape: 120 samples, 110 timesteps, 512 features

Test text shape: 31 samples, 110 timesteps, 100 features
Test audio shape: 31 samples, 110 timesteps, 100 features
Test visual shape: 31 samples, 110 timesteps, 512 features



In [61]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=int(d_model/num_heads))
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.mha(inputs, inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

class CrossModalAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.multi_head_attn = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
    
    def call(self, query, key_value):
        attn_output = self.multi_head_attn(query, key_value, key_value)
        return attn_output


class MultimodalModel(Model):
    def __init__(self, d_model, num_heads, dff, rate=0.1, num_classes=10):
        super(MultimodalModel, self).__init__()
        # Transformer block for processing video features
        self.video_transformer = TransformerBlock(d_model=512, num_heads=8, dff=dff, rate=rate)
        
        # Reducing dimensionality of processed video features to align with audio and text features
        self.dimensionality_reduction = Dense(d_model)
        
        # Text feature processing using bi-LSTM
        self.text_lstm = Bidirectional(LSTM(d_model, return_sequences=True))

        # Audio feature processing using 1D CNN
        self.audio_cnn = Conv1D(filters=d_model, kernel_size=3, activation='relu')

        # Cross-modal attention layers
        self.text_audio_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)
        self.text_video_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)

        # Final processing and classification layers
        self.concat = Concatenate()
        self.global_pool = GlobalAveragePooling1D()
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs, training):
        audio_features, text_features, video_features = inputs
        
        # Process video features and reduce dimensionality
        video_features_transformed = self.video_transformer(video_features, training=training)
        video_features_reduced = self.dimensionality_reduction(video_features_transformed)

        # Process text features
        text_features = self.text_lstm(text_features)
        print(text_features.shape)

        # Process audio features
        audio_features = self.audio_cnn(audio_features)
        print(audio_features.shape)
        
        # Apply cross-modal attention between audio-video and text-video
        text_audio_attn = self.text_audio_attention(text_features, audio_features)
        text_video_attn = self.text_video_attention(text_features, video_features_reduced)
        
        # Combine features from both attention mechanisms
        combined_features = self.concat([text_features, text_audio_attn, text_video_attn])
        print(combined_features.shape)
        outputs = self.classifier(combined_features)

        # Return the final prediction
        return outputs
    
    def evaluate(self, x_test, y_test):
        predictions = self.predict(x_test)
        y_pred = np.argmax(predictions, axis=-1).flatten()
        y_true = np.argmax(y_test, axis=-1).flatten()

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

        # Return a dictionary of metrics
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
        }


In [62]:
# Example instantiation and compilation
multimodal_model = MultimodalModel(d_model=100, num_heads=4, dff=2048, rate=0.1, num_classes=6)

# Compile the model
multimodal_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
multimodal_model.fit([train_audio, train_text, train_visual], train_label, epochs=10, batch_size=32)

# Evaluate the model
multimodal_model.evaluate([test_audio, test_text, test_visual], test_label)


Epoch 1/10
(None, 110, 600)
(None, 110, 600)


2024-03-30 16:48:15.442285: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-30 16:48:15.851566: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-30 16:48:15.956300: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-30 16:48:16.842938: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-30 16:48:16.878574: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 5s 542ms/step - loss: 1.3428 - accuracy: 0.4675
Epoch 2/10
4/4 [==============================] - 1s 295ms/step - loss: 0.9369 - accuracy: 0.7101
Epoch 3/10
4/4 [==============================] - 1s 287ms/step - loss: 0.4527 - accuracy: 0.9301
Epoch 4/10
4/4 [==============================] - 1s 263ms/step - loss: 0.2040 - accuracy: 0.9558
Epoch 5/10
4/4 [==============================] - 1s 272ms/step - loss: 0.1569 - accuracy: 0.9630
Epoch 6/10
4/4 [==============================] - 1s 265ms/step - loss: 0.1304 - accuracy: 0.9635
Epoch 7/10
4/4 [==============================] - 1s 285ms/step - loss: 0.1162 - accuracy: 0.9654
Epoch 8/10
4/4 [==============================] - 1s 246ms/step - loss: 0.1062 - accuracy: 0.9669
Epoch 9/10
4/4 [==============================] - 1s 250ms/step - loss: 0.0988 - accuracy: 0.9686
Epoch 10/10
4/4 [==============================] - 1s 245ms/step - loss: 0.0937 - accuracy: 0.9690
(None, 110, 600)


2024-03-30 16:48:29.375970: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-30 16:48:29.502123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-30 16:48:29.526062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 825ms/step


{'accuracy': 0.7917888563049853,
 'precision': 0.6727422557231058,
 'recall': 0.6462977881366591,
 'f1_score': 0.6498058877653472}